This is a page for exploring a single STAC collection

In [ ]:
import geoviews as gv
import panel as pn
import param

import utils.cards
import utils.location
import utils.template
import utils.xyt
from utils.cog import COGDataset
from utils.collection_types import map_collection_to_dataset
from utils.zarr import ZarrDataset

pn.extension()
gv.extension("bokeh")

In [ ]:
class Page(utils.location.UrlQueryParams):
    dataset: ZarrDataset | COGDataset | None = param.ClassSelector(
        class_=ZarrDataset | COGDataset, allow_None=True, default=None
    )  # type: ignore

    @param.depends("site", "collection", watch=True)
    def update_poi_dataset(self):
        if self.site is None or self.collection is None:
            self.dataset = None
            return
        self.dataset = map_collection_to_dataset(self.collection)

    @param.depends("site", "collection", "dataset", watch=False)
    def main(self) -> pn.Column:
        """content to render in the main area of a template"""
        elements = []

        if self.site is None:
            elements.append(pn.pane.Alert("Site not found", alert_type="danger"))
        elif self.collection is None:
            elements.append(pn.pane.Alert("Collection not found", alert_type="danger"))
        elif self.dataset is None:
            elements.append(pn.pane.Alert("Unable to parse collection", alert_type="danger"))
        else:
            site_name = self.site.title or self.site.id
            collection_name = self.collection.title or self.collection.id
            elements.append(
                pn.pane.Markdown(
                    f"""
                    # Data explorer

                    On this page you can interactively explore our data collection {collection_name}
                    for the site {site_name}.
                    """
                )
            )
            elements.append(pn.pane.Markdown("## Site"))
            elements.append(utils.cards.site(self.site, collapsed=True))
            elements.append(pn.pane.Markdown("## Collection"))
            elements.append(utils.cards.collection(self.site, self.collection, collapsed=True))
            elements.append(
                pn.pane.Markdown(
                    f"""
                    ## Data

                    This collection is in the EPSG:{self.dataset.crs.to_epsg()} coordinate reference system.
                    It is plotted in the native projection.

                    Use the controls in the sidebar to customise the plots.
                    Or click to select a point of interest.
                    """
                )
            )
            elements.append(self.dataset)

        return pn.Column(*elements)

    @param.depends("dataset", watch=False)
    def sidebar(self) -> pn.Column:
        """content to render in the sidebar of a template"""
        elements = []

        if self.dataset is not None:
            elements.append(self.dataset.location)
            elements.append(self.dataset.widgets())

        return pn.Column(*elements)

In [ ]:
page = Page()

In [ ]:
# In a notebook environment pn.state.location is not initialized until the first plot has been displayed

pn.state.location.sync(page, {"site_id": "site-id", "collection_id": "collection-id"})  # type: ignore

In [ ]:
template = utils.template.get_template(main=page.main, sidebar=page.sidebar)
template.servable();